# Práctica 1: Irene J. Ventura Farias

## Ejercicio 2: Hive

## Parte I

En este trabajo se van a utilizar ficheros dos ficheros que contienen los siguientes campos:
- __authors.csv__: author_id; author_name.
- __datasets.csv__: title; author_id; bestsellers-rank; imprint; publication-date;rating-avg; rating-count

Partiendo de este dataset, desarrolla las órdenes de HiveQL que implementen las
siguientes tareas.

In [1]:
#creamos un directorio para tratar los ficheros que vayamos generando y usando en local
! mkdir -p hive1
import os
os.chdir("hive1")

In [2]:
! pwd #Comprobamos que estamos en la ruta creada

/media/notebooks/hive1


In [4]:
!ls ../hive1 #Comprobamos que los ficheros estan cargados en local 

authors.csv  dataset.csv


In [5]:
 #Creamos los directorios en HDFS
! hadoop fs -mkdir /user/root/hive1 #Ruta principal
! hadoop fs -mkdir /user/root/hive1/authors #Ruta fichero authors.csv
! hadoop fs -mkdir /user/root/hive1/dataset #Ruta fichero dataset.csv

In [6]:
# Copiamos los ficheros a HDFS a sus respectivos directorios
! hadoop fs -put ../hive1/authors.csv /user/root/hive1/authors/
! hadoop fs -put ../hive1/dataset.csv /user/root/hive1/dataset/
# Confirmamos que se hayan cargado previamente 
! hadoop fs -ls /user/root/hive1/authors/
! hadoop fs -ls /user/root/hive1/dataset/

Found 1 items
-rw-r--r--   3 root supergroup     464036 2021-11-14 11:02 /user/root/hive1/authors/authors.csv
Found 1 items
-rw-r--r--   3 root supergroup    3391054 2021-11-14 11:02 /user/root/hive1/dataset/dataset.csv


__1. Crea las tablas necesarias para almacenar datos. Pueden ser internas o externas en función de los datos que se desee. La decisión de interna o externa debe estar razonada.__

He decidido realizar las dos tablas como externas, ya que el ejercicio se trata de hacer consultas de los datos y  no hay que  realizar modificaciones. Por lo que se hará referencia en la consulta sobre sus correspondientes archivos HDFS.

- Para ambas tablas se les ha asignado el parámetro *"skip.header.line.count"*, de forma que al cargar los datos sobre las tablas ignora las cabeceras de los ficheros.

In [8]:
%%writefile createHiveEj2.hql

-- Creamos la BD
CREATE DATABASE IF NOT EXISTS ej2_p1;

-- Nos aseguramos de usar la BD creada
USE ej2_p1;

-- Creamos las tablas externas 
-- **AUTORES**
CREATE EXTERNAL TABLE IF NOT EXISTS authors 
 (author_id  INT,   
  author_name  STRING) 
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\;' 
LOCATION '/user/root/hive1/authors'
TBLPROPERTIES ("skip.header.line.count"="1");

-- **LIBROS**
CREATE EXTERNAL TABLE IF NOT EXISTS dataset (
  title  STRING,  
  author_id  INT,
  bestseller_rank  INT,
  imprint  STRING,
  publication_date STRING,
  rating_avg  FLOAT,
  rating_count INT) 
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\;' 
LOCATION '/user/root/hive1/dataset'
TBLPROPERTIES ("skip.header.line.count"="1");

--Comprobamos que se hayan creado las tablas correctamente
SHOW TABLES;
DESCRIBE authors;
DESCRIBE dataset;

Overwriting createHiveEj2.hql


In [9]:
! beeline -u "jdbc:hive2://localhost:10000/" -f createHiveEj2.hql

Connecting to jdbc:hive2://localhost:10000/
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://localhost:10000/> 
0: jdbc:hive2://localhost:10000/> -- Creamos la BD
0: jdbc:hive2://localhost:10000/> CREATE DATABASE IF NOT EXISTS ej2_p1;
INFO  : Compiling command(queryId=root_20211114111528_33bd6692-96de-4a2c-a9e1-48e9e9141b95): CREATE DATABASE IF NOT EXISTS ej2_p1
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20211114111528_33bd6692-96de-4a2c-a9e1-48e9e9141b95); Time taken: 5.064 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20211114111528_33bd6692-96de-4a2c-a9e1-48e9e9141b95): CREATE DATABASE IF NOT EXISTS ej2_p1
INFO  : Starting task 

__2. Importa los datos en las tablas creadas.__

Como hemos hecho copia de los csvs a HDFS  previamente, al crear las tablas y asignarle en LOCATION las rutas de HDFS los datos de los csv, se importaron automaticamente. Es por eso que simplemente realizamos consultas simples sobre las tablas a modo de confirmación.

In [10]:
! beeline -u "jdbc:hive2://localhost:10000/ej2_p1" -e 'SELECT * FROM authors LIMIT 5;'

Connecting to jdbc:hive2://localhost:10000/ej2_p1
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211114111648_a88857b0-1b26-4127-9461-925076fc957f): SELECT * FROM authors LIMIT 5
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:authors.author_id, type:int, comment:null), FieldSchema(name:authors.author_name, type:string, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20211114111648_a88857b0-1b26-4127-9461-925076fc957f); Time taken: 14.026 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20211114111648_a88857b0-1b26-4127-9461-925076fc957f): SELECT * FROM authors LIMIT 5
INFO  : Completed executing command(queryId=root_20211114111648_

In [11]:
! beeline -u "jdbc:hive2://localhost:10000/ej2_p1" -e 'SELECT * FROM dataset LIMIT 5;'

Connecting to jdbc:hive2://localhost:10000/ej2_p1
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211114111826_c5fe563e-cb89-49c9-aceb-e0c4c5d6915e): SELECT * FROM dataset LIMIT 5
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:dataset.title, type:string, comment:null), FieldSchema(name:dataset.author_id, type:int, comment:null), FieldSchema(name:dataset.bestseller_rank, type:int, comment:null), FieldSchema(name:dataset.imprint, type:string, comment:null), FieldSchema(name:dataset.publication_date, type:string, comment:null), FieldSchema(name:dataset.rating_avg, type:float, comment:null), FieldSchema(name:dataset.rating_count, type:int, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_202111141118

__3. Crea una vista sobre las tablas creadas. Esta vista tendrá para cada título, el nombre del autor, fecha de publicación, y valoración media.__

In [12]:
%%writefile view1Ej2.hql

-- Creamos las vista
CREATE VIEW IF NOT EXISTS vista1_ej21 
AS SELECT d.title, a.author_name, d.publication_date, d.rating_avg 
FROM authors a JOIN dataset d 
ON a.author_id = d.author_id;

-- Comprobar que se ha creado la vista 
SHOW VIEWS;

Writing view1Ej2.hql


In [13]:
! beeline -u "jdbc:hive2://localhost:10000/ej2_p1" -f view1Ej2.hql

Connecting to jdbc:hive2://localhost:10000/ej2_p1
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://localhost:10000/ej2_p1> 
0: jdbc:hive2://localhost:10000/ej2_p1> -- Creamos las vista
0: jdbc:hive2://localhost:10000/ej2_p1> CREATE VIEW IF NOT EXISTS vista1_ej21 
ication_d . . . . . . . . . . . . . . > AS SELECT d.title, a.author_name, d.publ ate, d.rating_avg 
. . . . . . . . . . . . . . . . . . . > FROM authors a JOIN dataset d 
. . . . . . . . . . . . . . . . . . . > ON a.author_id = d.author_id;
INFO  : Compiling command(queryId=root_20211114112054_addb3038-0218-41c7-b77d-3980f4b6edb3): CREATE VIEW IF NOT EXISTS vista1_ej21
AS SELECT d.title, a.author_name, d.publication_date, d.rating_avg
FROM authors a JOIN dataset d
ON a.author_id = d.author_id
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive s

In [14]:
! beeline -u "jdbc:hive2://localhost:10000/ej2_p1" -e 'SELECT * FROM vista1_ej21 LIMIT 5;'

Connecting to jdbc:hive2://localhost:10000/ej2_p1
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211114112220_3e2c389d-fb66-4dc3-b098-ee85fc7abba0): SELECT * FROM vista1_ej21 LIMIT 5
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:vista1_ej21.title, type:string, comment:null), FieldSchema(name:vista1_ej21.author_name, type:string, comment:null), FieldSchema(name:vista1_ej21.publication_date, type:string, comment:null), FieldSchema(name:vista1_ej21.rating_avg, type:float, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20211114112220_3e2c389d-fb66-4dc3-b098-ee85fc7abba0); Time taken: 3.532 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=

__4.1. Crea las consultas de Hive necesarias para responder las siguientes cuestiones:__

    - ¿Cuál es el título del libro con mejor valoración media del autor Rand McNally?
    

In [19]:
#Imprime el primer libro que hay en la bd que tenga rating5 y sea de Rand McNally
# ya que este autor tiene más de un libro con esa valoración
! beeline -u "jdbc:hive2://localhost:10000/ej2_p1" -e "\
SELECT title \
FROM vista1_ej21 \
WHERE author_name = 'Rand McNally'\
ORDER BY rating_avg DESC\
LIMIT 1;"

Connecting to jdbc:hive2://localhost:10000/ej2_p1
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211114114341_3b2d5877-e8d1-44a7-a7e0-9b67a8691f5d): SELECT title  FROM vista1_ej21  WHERE author_name = 'Rand McNally' ORDER BY rating_avg DESC LIMIT 1
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:title, type:string, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20211114114341_3b2d5877-e8d1-44a7-a7e0-9b67a8691f5d); Time taken: 3.146 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20211114114341_3b2d5877-e8d1-44a7-a7e0-9b67a8691f5d): SELECT title  FROM vista1_ej21  WHERE author_name = 'Rand McNally' ORDER BY rating_avg DESC LIMIT 1


__4.2. Crea las consultas de Hive necesarias para responder las siguientes cuestiones:__

    - ¿Cuáles son los cinco autores cuyos libros tienen más valoraciones?

In [16]:
%%writefile view2Ej2.hql

-- Creamos las vista
CREATE VIEW IF NOT EXISTS vista2_ej21
    AS SELECT d.title, a.author_name, d.rating_count
    FROM authors a JOIN dataset d 
    ON a.author_id = d.author_id;

-- Consultar que se ha creado la vista 
SHOW VIEWS;

Writing view2Ej2.hql


In [17]:
! beeline -u "jdbc:hive2://localhost:10000/ej2_p1" -f view2Ej2.hql

Connecting to jdbc:hive2://localhost:10000/ej2_p1
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://localhost:10000/ej2_p1> 
0: jdbc:hive2://localhost:10000/ej2_p1> -- Creamos las vista
0: jdbc:hive2://localhost:10000/ej2_p1> CREATE VIEW IF NOT EXISTS vista2_ej21
rating_count. . . . . . . . . . . . . >     AS SELECT d.title, a.author_name, d. 
. . . . . . . . . . . . . . . . . . . >     FROM authors a JOIN dataset d 
. . . . . . . . . . . . . . . . . . . >     ON a.author_id = d.author_id;
INFO  : Compiling command(queryId=root_20211114113525_0a604559-f345-47f6-92b0-9274fc3f94db): CREATE VIEW IF NOT EXISTS vista2_ej21
AS SELECT d.title, a.author_name, d.rating_count
FROM authors a JOIN dataset d
ON a.author_id = d.author_id
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[F

In [18]:
! beeline -u "jdbc:hive2://localhost:10000/ej2_p1" -e "\
SELECT author_name\
FROM vista2_ej21 \
GROUP BY author_name \
ORDER BY SUM(rating_count) DESC\
LIMIT 5;"

Connecting to jdbc:hive2://localhost:10000/ej2_p1
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211114113702_618ffe69-4652-4923-8417-cfa960fd2b99): SELECT author_name FROM vista2_ej21  GROUP BY author_name  ORDER BY SUM(rating_count) DESC LIMIT 5
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:author_name, type:string, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20211114113702_618ffe69-4652-4923-8417-cfa960fd2b99); Time taken: 4.002 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20211114113702_618ffe69-4652-4923-8417-cfa960fd2b99): SELECT author_name FROM vista2_ej21  GROUP BY author_name  ORDER BY SUM(rating_count) DESC LIMI

__4.3. Crea las consultas de Hive necesarias para responder las siguientes cuestiones:__

    - ¿Cuáles son los dos libros más recientes del autor Rand McNally?
    
En este caso como hay que ordenar por fecha y este campo lo he tratado como un string, hice la conversion del publication_day para el Order By 

In [20]:
! beeline -u "jdbc:hive2://localhost:10000/ej2_p1" -e "\
SELECT title, publication_date\
FROM vista1_ej21 \
WHERE author_name = 'Rand McNally'\
ORDER BY FROM_UNIXTIME(UNIX_TIMESTAMP(publication_date, 'dd/MM/yyyy')) DESC\
LIMIT 2"

Connecting to jdbc:hive2://localhost:10000/ej2_p1
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211114115036_084a398b-4566-4235-afbd-f8374ee81363): SELECT title, publication_date FROM vista1_ej21  WHERE author_name = 'Rand McNally' ORDER BY FROM_UNIXTIME(UNIX_TIMESTAMP(publication_date, 'dd/MM/yyyy')) DESC LIMIT 2
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:title, type:string, comment:null), FieldSchema(name:publication_date, type:string, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20211114115036_084a398b-4566-4235-afbd-f8374ee81363); Time taken: 3.267 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20211114115036_084a398b

## Ejercicio Hive
## Parte 2

Para este apartado he decidido usar el dataset de el Top 100 de la Lista Forbes (https://www.kaggle.com/darinhawley/forbes-celebrity-100-19992020)

Este dataset consta de un solo fichero, el cual contiene 4 campos:
- Name
- Ocuppation
- Rank
- Year

In [1]:
#creamos un directorio para tratar los ficheros que vayamos generando y usando en local
! mkdir -p hive2
import os
os.chdir("hive2")

In [2]:
! pwd #Comprobamos que estamos en la ruta creada

/media/notebooks/hive2


In [3]:
!ls ../hive2 #Comprobamos que los ficheros estan cargados en local 

forbescelebrities.csv


In [4]:
 #Creamos los directorios en HDFS
! hadoop fs -mkdir /user/root/hive2 #Ruta principal
! hadoop fs -mkdir /user/root/hive2/forbescelebrities #Ruta fichero authors.csv

In [5]:
# Copiamos los ficheros a HDFS a sus respectivos directorios
! hadoop fs -put ../hive2/forbescelebrities.csv /user/root/hive2/forbescelebrities/
# Confirmamos que se hayan cargado previamente 
! hadoop fs -ls /user/root/hive2/forbescelebrities/

Found 1 items
-rw-r--r--   3 root supergroup      72933 2021-11-14 12:01 /user/root/hive2/forbescelebrities/forbescelebrities.csv


In [6]:
%%writefile createHiveEjP2.hql

-- Creamos la BD
CREATE DATABASE IF NOT EXISTS ej2_p2;

-- Nos aseguramos de usar la BD creada
USE ej2_p2;

-- Creamos las tablas externas 
-- **CELEBRIDADES**
CREATE EXTERNAL TABLE IF NOT EXISTS celebrities 
 (Name  STRING,   
  Year  INT,
  Rank INT,
  Occuppation STRING) 
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\,' 
LOCATION '/user/root/hive2/forbescelebrities'
TBLPROPERTIES ("skip.header.line.count"="1");

--Comprobamos que se hayan creado las tablas correctamente
SHOW TABLES;
DESCRIBE celebrities;

Writing createHiveEjP2.hql


In [7]:
! beeline -u "jdbc:hive2://localhost:10000/" -f createHiveEjP2.hql

Connecting to jdbc:hive2://localhost:10000/
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://localhost:10000/> 
0: jdbc:hive2://localhost:10000/> -- Creamos la BD
0: jdbc:hive2://localhost:10000/> CREATE DATABASE IF NOT EXISTS ej2_p2;
INFO  : Compiling command(queryId=root_20211114120323_8969d9c5-049e-4598-92bf-32b432f41252): CREATE DATABASE IF NOT EXISTS ej2_p2
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20211114120323_8969d9c5-049e-4598-92bf-32b432f41252); Time taken: 0.461 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20211114120323_8969d9c5-049e-4598-92bf-32b432f41252): CREATE DATABASE IF NOT EXISTS ej2_p2
INFO  : Starting task 

In [8]:
#Comprobamos que la tabla se haya cargado correctamente
! beeline -u "jdbc:hive2://localhost:10000/ej2_p2" -e "SELECT * FROM celebrities LIMIT 5;"

Connecting to jdbc:hive2://localhost:10000/ej2_p2
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211114120453_6142d29d-3e1d-4368-8f3a-32310e66b820): SELECT * FROM celebrities LIMIT 5
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:celebrities.name, type:string, comment:null), FieldSchema(name:celebrities.year, type:int, comment:null), FieldSchema(name:celebrities.rank, type:int, comment:null), FieldSchema(name:celebrities.occuppation, type:string, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20211114120453_6142d29d-3e1d-4368-8f3a-32310e66b820); Time taken: 1.648 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20211114120453_6142

__Consulta 1: ¿Cuantos Deportistas hay en el top 100 cada año?Ordenados desde la actualidad al pasado__

In [9]:
! beeline -u "jdbc:hive2://localhost:10000/ej2_p2" -e "\
 SELECT COUNT(occuppation) AS count_players, year \
 FROM celebrities \
 WHERE occuppation LIKE '%Player' \
 GROUP BY year ORDER BY year DESC;"

Connecting to jdbc:hive2://localhost:10000/ej2_p2
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211114120529_3d030777-f0c3-4abd-a9d9-20b2d976ef91): SELECT COUNT(occuppation) AS count_players, year   FROM celebrities   WHERE occuppation LIKE '%Player'   GROUP BY year ORDER BY year DESC
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:count_players, type:bigint, comment:null), FieldSchema(name:year, type:int, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20211114120529_3d030777-f0c3-4abd-a9d9-20b2d976ef91); Time taken: 1.912 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20211114120529_3d030777-f0c3-4abd-a9d9-20b2d976ef91): SELECT

__Consulta 2: Consultar cual es la Ocupación más mencionada y en que año__


In [10]:
! beeline -u "jdbc:hive2://localhost:10000/ej2_p2" -e "\
SELECT occuppation, year, COUNT(occuppation) AS count_occuppation \
FROM celebrities \
GROUP BY occuppation, year\
ORDER BY count_occuppation DESC\
LIMIT 1;"

Connecting to jdbc:hive2://localhost:10000/ej2_p2
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211114121905_18ee2dfd-a0cb-4735-9d4b-974865babc69): SELECT occuppation, year, COUNT(occuppation) AS count_occuppation  FROM celebrities  GROUP BY occuppation, year ORDER BY count_occuppation DESC LIMIT 1
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:occuppation, type:string, comment:null), FieldSchema(name:year, type:int, comment:null), FieldSchema(name:count_occuppation, type:bigint, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20211114121905_18ee2dfd-a0cb-4735-9d4b-974865babc69); Time taken: 1.404 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing comma

__Consulta 3: Cuales son las ocupaciones relacionadas con la TV__

In [11]:
! beeline -u "jdbc:hive2://localhost:10000/ej2_p2" -e "\
SELECT DISTINCT occuppation\
FROM celebrities\
WHERE occuppation LIKE '%TV%' ORDER BY occuppation"

Connecting to jdbc:hive2://localhost:10000/ej2_p2
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211114122436_8e32ba47-adbc-421f-8786-d51aae1e58d2): SELECT DISTINCT occuppation FROM celebrities WHERE occuppation LIKE '%TV%' ORDER BY occuppation
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:occuppation, type:string, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20211114122436_8e32ba47-adbc-421f-8786-d51aae1e58d2); Time taken: 1.737 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20211114122436_8e32ba47-adbc-421f-8786-d51aae1e58d2): SELECT DISTINCT occuppation FROM celebrities WHERE occuppation LIKE '%TV%' ORDER BY occuppation
WA

__Consulta 4: Top 5 de las celebridades que mas veces aparece en la lista__

In [12]:
! beeline -u "jdbc:hive2://localhost:10000/ej2_p2" -e "\
 SELECT COUNT(name) AS count_name, name\
 FROM celebrities \
 GROUP BY name \
 ORDER BY count_name DESC\
 LIMIT 5;"

Connecting to jdbc:hive2://localhost:10000/ej2_p2
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211114123412_8a1719e5-9b1c-41d7-8f2f-364cb829d722): SELECT COUNT(name) AS count_name, name  FROM celebrities   GROUP BY name   ORDER BY count_name DESC  LIMIT 5
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:count_name, type:bigint, comment:null), FieldSchema(name:name, type:string, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20211114123412_8a1719e5-9b1c-41d7-8f2f-364cb829d722); Time taken: 1.199 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20211114123412_8a1719e5-9b1c-41d7-8f2f-364cb829d722): SELECT COUNT(name) AS count_name, n